베이지안 이론

In [1]:
import nltk

In [2]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [5]:
import os

os.chdir('/Users/oyujeong/Desktop/mulcam_ujeong/data')

In [4]:
import numpy as np
import pandas as pd

# corpos: 말뭉치
# stopwords: 정지어(금지어) -> (거를 단어들, 특수문자, 관사)
from nltk.corpus import stopwords
import string
import nltk


In [12]:
# spam - spam.csv
raw = pd.read_csv("/Users/oyujeong/Desktop/mulcam_ujeong/data/spam - spam.csv")
raw.head()

,Unnamed: 0,num,label,text,label_num
0,0,1,spam,your free lottery,1
1,1,2,spam,free lottery free you,1
2,2,3,ham,your free apple,0
3,3,4,ham,free to contact me,0
4,4,5,ham,I won award,0


In [13]:
# raw 데이터 보존
df = raw.copy()
df[:1]

,Unnamed: 0,num,label,text,label_num
0,0,1,spam,your free lottery,1


In [24]:
# 텍스트 데이터 전처리 함수 생성

def process_text(text):
    # text 에서 구두점 삭제
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    # text 에서 불용어(무의미한 단어 -> 접미사, 조사 등) 삭제
    # 불용어 제거 >> 소문자로 통일
    cleaned_words = [word for word in nopunc.split() if word.lower() not in stopwords.words("english")]
    
    return cleaned_words

In [18]:
df.columns

Index(['Unnamed: 0', 'num', 'label', 'text', 'label_num'], dtype='object')

In [19]:
df['text']
# Series

0        your free lottery
1    free lottery free you
2          your free apple
3       free to contact me
4              I won award
5        my lottery ticket
Name: text, dtype: object

In [25]:
df['text'].apply(process_text)

0          [free, lottery]
1    [free, lottery, free]
2            [free, apple]
3          [free, contact]
4                  [award]
5        [lottery, ticket]
Name: text, dtype: object

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

# bow: Bag Of Words
messages_bow = CountVectorizer(analyzer=process_text).fit_transform(df['text'])

In [28]:
messages_bow

<6x6 sparse matrix of type '<class 'numpy.int64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [ ]:
# 모델링
# 모델링 준비 단계 (데이터 셋 분리)
#  >> 훈련용 데이터, 평가용(테스트용) 데이터 구분
#  >> 일반적으로 8:2 or 7:3 
#  >> 훈련용 데이터 (훈련용 데이터 / 검증용 데이터), 평가용 데이터
#     >> train data, validation data, test data

In [29]:
df['label_num']

0    1
1    1
2    0
3    0
4    0
5    1
Name: label_num, dtype: int64

In [33]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test =\
train_test_split(messages_bow, df['label_num'], test_size=0.2, random_state=42)

In [34]:
# 모델링
# 나이브 베이즈

from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()

# 훈련용 데이터로만 >> 모델 생성
clf.fit(x_train, y_train)


MultinomialNB()

In [35]:
# 모델 사용 > 예측

clf.predict(x_train)

array([1, 0, 0, 0])

In [38]:
y_train
print(y_train.values)

[1 0 0 0]


평가지표 

In [39]:
# 학습 데이터셋에서 모델의 정확도 표현

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score


In [41]:
pred_train = clf.predict(x_train)

In [42]:
# metrics pkg >> 정밀도, 재현률, f1 score 구하기

print(classification_report(y_train, pred_train))
# y_train >> y값, pred_train: train data 로 예측한 y_hat

# 정밀도 (precision): 예측모델이 실제값을 잘 예측해서 분류하는 것
# 재현률 (recall): 실제값을 예측모델이 잘 예측해서 분류하는 것
# f1 score: 정밀도, 재현률이 (하도 중요해서) 이 둘을 조화 평균한 것

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         1

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4



In [45]:
# 혼동행렬로 표현

print(f'혼동행렬: \n{confusion_matrix(y_train, pred_train)}\n')
print(f'정확도(accuracy): \n{accuracy_score(y_train, pred_train)}')

혼동행렬: 
[[3 0]
 [0 1]]

정확도(accuracy): 
1.0


In [46]:
# 평가용 데이터(test data) >> 모델 정확도 테스트

print(f'예측값: {clf.predict(x_test)}')


예측값: [0 0]


In [48]:
y_test

0    1
1    1
Name: label_num, dtype: int64

In [47]:
print(f'실제값: {y_test.values}')

실제값: [1 1]


In [50]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score

pred_test = clf.predict(x_test)
pred_test

array([0, 0])

In [54]:
print(classification_report(y_test, pred_test))
print(f'\n혼동행렬: \n{confusion_matrix(y_test, pred_test)}\n')
print(f'정확도: {accuracy_score(y_test, pred_test)}')

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       2.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0


혼동행렬: 
[[0 0]
 [2 0]]

정확도: 0.0


/Users/oyujeong/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/oyujeong/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/oyujeong/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/oyujeong/anaconda3/l